In [1]:
import numpy as np 
from sklearn import preprocessing

raw_csv_data = np.loadtxt('rec/Audiobooks_data.csv', delimiter= ',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

In [2]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equals_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

In [3]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equals_priors)

In [4]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [5]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1820.0 3579 0.508521933500978
200.0 447 0.44742729306487694
217.0 448 0.484375


In [6]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [7]:
import tensorflow as tf

In [8]:
npz = np.load('Audiobooks_data_train.npz')
ml_train_inputs = npz['inputs'].astype(np.float)
ml_train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
ml_validation_inputs, ml_validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
ml_test_inputs, ml_test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

In [9]:
 input_size = 10
 output_size = 2
 hidden_layer_size = 50

 model = tf.keras.Sequential([
     tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
     tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
     tf.keras.layers.Dense(output_size, activation='softmax')
 ])

 model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

 batch_size = 100
 max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

 model.fit(ml_train_inputs,
 ml_train_targets,
 batch_size = batch_size,
 epochs= max_epochs,
 validation_data=(ml_validation_inputs, ml_validation_targets),
 verbose = 2,
 callbacks = [early_stopping]
 )

Epoch 1/100
36/36 - 4s - loss: 0.6020 - accuracy: 0.6980 - val_loss: 0.4139 - val_accuracy: 0.8926
Epoch 2/100
36/36 - 0s - loss: 0.3818 - accuracy: 0.8662 - val_loss: 0.3064 - val_accuracy: 0.9038
Epoch 3/100
36/36 - 0s - loss: 0.3303 - accuracy: 0.8773 - val_loss: 0.2803 - val_accuracy: 0.9060
Epoch 4/100
36/36 - 0s - loss: 0.3069 - accuracy: 0.8860 - val_loss: 0.2622 - val_accuracy: 0.9060
Epoch 5/100
36/36 - 0s - loss: 0.2910 - accuracy: 0.8919 - val_loss: 0.2534 - val_accuracy: 0.9128
Epoch 6/100
36/36 - 0s - loss: 0.2785 - accuracy: 0.8927 - val_loss: 0.2463 - val_accuracy: 0.9128
Epoch 7/100
36/36 - 0s - loss: 0.2707 - accuracy: 0.8983 - val_loss: 0.2414 - val_accuracy: 0.9128
Epoch 8/100
36/36 - 0s - loss: 0.2625 - accuracy: 0.8994 - val_loss: 0.2364 - val_accuracy: 0.9150
Epoch 9/100
36/36 - 0s - loss: 0.2600 - accuracy: 0.9025 - val_loss: 0.2319 - val_accuracy: 0.9128
Epoch 10/100
36/36 - 0s - loss: 0.2573 - accuracy: 0.9022 - val_loss: 0.2298 - val_accuracy: 0.9172
Epoch 11/

In [11]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.2581 - accuracy: 0.8951


In [12]:
print('\nTeste loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))


Teste loss: 0.26. Test accuracy: 89.51%


In [ ]:
 #Kaggle